<a href="https://www.kaggle.com/code/averma111/playground-s3e12-xgboost-lgbm-catboost?scriptVersionId=124994487" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostClassifier, Pool
import warnings
warnings.filterwarnings('ignore')

In [3]:
!wget http://bit.ly/3ZLyF82 -O CSS.css -q
    
from IPython.core.display import HTML
with open('./CSS.css', 'r') as file:
    custom_css = file.read()

HTML(custom_css)

In [12]:
Root_path ='/kaggle/input/playground-series-s3e12'
Root_original ='/kaggle/input/kidney-stone-prediction-based-on-urine-analysis'

original = pd.read_csv(Root_original+'/kindey stone urine analysis.csv')
train = pd.read_csv(Root_path+'/train.csv')
test = pd.read_csv(Root_path+'/test.csv')

In [5]:
train.head(5)

,id,gravity,ph,osmo,cond,urea,calc,target
0,0,1.013,6.19,443,14.8,124,1.45,0
1,1,1.025,5.40,703,23.6,394,4.18,0
2,2,1.009,6.13,371,24.5,159,9.04,0
3,3,1.021,4.91,442,20.8,398,6.63,1
4,4,1.021,5.53,874,17.8,385,2.21,1


In [6]:
test.head(5)

,id,gravity,ph,osmo,cond,urea,calc
0,414,1.017,5.24,345,11.5,152,1.16
1,415,1.020,5.68,874,29.0,385,3.46
2,416,1.024,5.36,698,19.5,354,13.00
3,417,1.020,5.33,668,25.3,252,3.46
4,418,1.011,5.87,567,29.0,457,2.36


In [7]:
orignal.head(5)

,gravity,ph,osmo,cond,urea,calc,target
0,1.021,4.91,725,14.0,443,2.45,0
1,1.017,5.74,577,20.0,296,4.49,0
2,1.008,7.20,321,14.9,101,2.36,0
3,1.011,5.51,408,12.6,224,2.15,0
4,1.005,6.52,187,7.5,91,1.16,0


In [8]:
def summary(text, df):
    print(f'{text} shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    return summ

In [9]:
summary('train',train)

train shape: (414, 8)


,dtypes,null,unique,min,median,max,mean,std
id,int64,0,414,0.000,206.500,413.00,206.500000,119.655756
gravity,float64,0,28,1.005,1.018,1.04,1.017894,0.006675
ph,float64,0,71,4.760,5.740,7.94,5.955459,0.642260
osmo,int64,0,80,187.000,679.500,1236.00,651.545894,234.676567
cond,float64,0,65,5.100,22.200,38.00,21.437923,7.514750
urea,int64,0,73,10.000,277.000,620.00,278.657005,136.442249
calc,float64,0,85,0.170,3.130,14.34,4.114638,3.217641
target,int64,0,2,0.000,0.000,1.00,0.444444,0.497505


In [10]:
summary('test',test)

test shape: (276, 7)


,dtypes,null,unique,min,median,max,mean,std
id,int64,0,276,414.000,551.500,689.000,551.500000,79.818544
gravity,float64,0,27,1.005,1.019,1.034,1.018330,0.006569
ph,float64,0,62,4.900,5.730,7.940,5.939312,0.607143
osmo,int64,0,75,187.000,658.000,1236.000,654.891304,236.361932
cond,float64,0,58,5.100,23.600,38.000,21.886594,7.346069
urea,int64,0,71,64.000,282.000,620.000,280.985507,134.877856
calc,float64,0,74,0.170,3.230,14.340,4.076848,3.088524


In [13]:
summary('original',original)

original shape: (79, 7)


,dtypes,null,unique,min,median,max,mean,std
gravity,float64,0,29,1.005,1.018,1.04,1.018114,0.007239
ph,float64,0,70,4.760,5.940,7.94,6.028481,0.724307
osmo,int64,0,77,187.000,594.000,1236.00,612.848101,237.514755
cond,float64,0,64,5.100,21.400,38.00,20.813924,7.938994
urea,int64,0,73,10.000,260.000,620.00,266.405063,131.254550
calc,float64,0,75,0.170,3.160,14.34,4.138987,3.260051
target,int64,0,2,0.000,0.000,1.00,0.430380,0.498293
